<a href="https://colab.research.google.com/github/brennanMosher/False-Data-Injection-Detection-for-Smart-Grid-Systems/blob/main/CISC_850_project_feature_selection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/brennanMosher/False-Data-Injection-Detection-for-Smart-Grid-Systems.git

Cloning into 'False-Data-Injection-Detection-for-Smart-Grid-Systems'...
remote: Enumerating objects: 96, done.
remote: Counting objects: 100% (96/96), done.
remote: Compressing objects: 100% (78/78), done.
remote: Total 96 (delta 35), reused 66 (delta 14), pack-reused 0
Unpacking objects: 100% (96/96), done.
Checking out files: 100% (28/28), done.


In [72]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.feature_selection import SelectKBest, SelectPercentile
from sklearn.feature_selection import f_classif
from sklearn.feature_selection import mutual_info_classif, mutual_info_regression
from sklearn.feature_selection import chi2
from sklearn import utils
from matplotlib import pyplot

Read the CSV data

In [ ]:
training_dataset = pd.read_csv('training_data_injection_and_normal_events_dataset.csv')
testing_dataset = pd.read_csv('testing_data_injection_and_normal_events_dataset.csv')


##Replace inf values with 0
training_dataset.replace([np.inf, -np.inf], np.nan, inplace=True)
testing_dataset.replace([np.inf, -np.inf], np.nan, inplace=True)


In [ ]:
testing_dataset.head()
training_dataset.head()

,Unnamed: 0,R1-PA1:VH,R1-PM1:V,R1-PA2:VH,R1-PM2:V,R1-PA3:VH,R1-PM3:V,R1-PA4:IH,R1-PM4:I,R1-PA5:IH,...,control_panel_log4,relay1_log,relay2_log,relay3_log,relay4_log,snort_log1,snort_log2,snort_log3,snort_log4,marker
0,22122,151.008757,131534.3744,31.019935,130907.5427,-88.974616,131584.5210,144.551522,472.24069,26.510757,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,26934,152.246345,131910.4735,32.274713,131283.6417,-87.731298,131960.6200,151.071782,353.03608,31.518408,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,10625,-170.254409,131734.9606,69.780530,131709.8873,-50.225480,131810.1804,-169.830420,359.81115,67.706423,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,26778,-174.041660,134217.2143,65.987549,134192.1410,-54.007002,134292.4341,-174.293761,366.40311,70.015443,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2558,39.058533,132361.7923,-80.924559,131709.8873,159.081732,132411.9389,46.690331,242.80386,-78.707212,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [ ]:

training_dataset.fillna(training_dataset.mean(),inplace = True)
testing_dataset.fillna(testing_dataset.mean(),inplace = True)
X_train = training_dataset.iloc[:, 0:129].values
X_test = testing_dataset.iloc[:, 0:129].values
y_train = training_dataset.iloc[:, 129].values
y_test = testing_dataset.iloc[:, 129].values

In [82]:
def select_features(X_train, y_train, X_test):
	# configure to select all features
	fs = SelectKBest(score_func=f_classif, k='all')
	# learn relationship from training data
	fs.fit(X_train, y_train)
	# transform train input data
	X_train_fs = fs.transform(X_train)
	# transform test input data
	X_test_fs = fs.transform(X_test)
	return X_train_fs, X_test_fs, fs

In [83]:
# feature selection
X_train_fs, X_test_fs, fs = select_features(X_train, y_train, X_test)
# what are scores for the features
for i in range(len(fs.scores_)):
	print('Feature %d: %f' % (i, fs.scores_[i]))

Feature 0: 16183.865410
Feature 1: 11.686007
Feature 2: 1.548354
Feature 3: 8.358491
Feature 4: 0.644822
Feature 5: 7.260599
Feature 6: 1.760554
Feature 7: 6.347950
Feature 8: 2.105112
Feature 9: 14.086026
Feature 10: 7.260402
Feature 11: 5.796031
Feature 12: 10.696382
Feature 13: 11.653917
Feature 14: 0.021696
Feature 15: 0.294777
Feature 16: 10.232374
Feature 17: 0.408451
Feature 18: 13.498170
Feature 19: 7.177511
Feature 20: 7.185184
Feature 21: 2.647127
Feature 22: 8.656040
Feature 23: 7.348957
Feature 24: 16.160881
Feature 25: 4.928076
Feature 26: 4.136872
Feature 27: 0.259632
Feature 28: 2.207338
Feature 29: 1.806844
Feature 30: 1.398910
Feature 31: 1.281591
Feature 32: 8.598908
Feature 33: 1.535665
Feature 34: 1.360873
Feature 35: 2.713343
Feature 36: 0.731956
Feature 37: 4.990214
Feature 38: 3.857844
Feature 39: 7.282289
Feature 40: 28.414829
Feature 41: 10.784089
Feature 42: 1.426676
Feature 43: 0.171771
Feature 44: 0.549009
Feature 45: 8.077968
Feature 46: 0.423629
Feature 47

/usr/local/lib/python3.7/dist-packages/sklearn/feature_selection/_univariate_selection.py:112: UserWarning: Features [125 126] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/feature_selection/_univariate_selection.py:113: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


In [84]:
def select_features(X_train, y_train, X_test):
	# configure to select all features
	fs = SelectKBest(score_func=mutual_info_regression, k='all')
	# learn relationship from training data
	fs.fit(X_train, y_train)
	# transform train input data
	X_train_fs = fs.transform(X_train)
	# transform test input data
	X_test_fs = fs.transform(X_test)
	return X_train_fs, X_test_fs, fs


In [85]:
# feature selection
X_train_fs, X_test_fs, fs = select_features(X_train, y_train, X_test)
# what are scores for the features
for i in range(len(fs.scores_)):
	print('Feature %d: %f' % (i, fs.scores_[i]))

Feature 0: 0.608606
Feature 1: 0.035376
Feature 2: 0.018964
Feature 3: 0.038870
Feature 4: 0.013145
Feature 5: 0.036223
Feature 6: 0.013779
Feature 7: 0.028167
Feature 8: 0.030026
Feature 9: 0.029322
Feature 10: 0.032967
Feature 11: 0.024626
Feature 12: 0.022187
Feature 13: 0.037021
Feature 14: 0.013640
Feature 15: 0.002787
Feature 16: 0.013069
Feature 17: 0.000000
Feature 18: 0.003054
Feature 19: 0.023136
Feature 20: 0.023402
Feature 21: 0.007233
Feature 22: 0.003410
Feature 23: 0.004937
Feature 24: 0.005542
Feature 25: 0.000854
Feature 26: 0.000000
Feature 27: 0.017550
Feature 28: 0.010810
Feature 29: 0.008207
Feature 30: 0.040998
Feature 31: 0.032503
Feature 32: 0.038718
Feature 33: 0.026663
Feature 34: 0.042131
Feature 35: 0.026546
Feature 36: 0.033143
Feature 37: 0.027108
Feature 38: 0.038623
Feature 39: 0.053387
Feature 40: 0.026590
Feature 41: 0.043060
Feature 42: 0.041388
Feature 43: 0.024301
Feature 44: 0.000000
Feature 45: 0.000296
Feature 46: 0.000000
Feature 47: 0.000000
Fe